In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import BorderlineSMOTE

In [2]:
data_comments = pd.read_csv('./Dataset/data_comments_processed.csv')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data_comments['Comment'], data_comments['Sentiment'], test_size=0.1, random_state=42)

In [4]:
encoder = LabelEncoder()
encoder.fit(y_train)

LabelEncoder()

In [5]:
y_train_encoded = encoder.transform(y_train)

In [6]:
tfidf = TfidfVectorizer(max_features=60000, ngram_range=(1,2))
tfidf.fit(X_train)

TfidfVectorizer(max_features=60000, ngram_range=(1, 2))

In [7]:
X_train_tfidf = tfidf.transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [8]:
model_1 = LogisticRegression(solver='liblinear', random_state=42)
model_2 = MultinomialNB(force_alpha=True)

In [18]:
model_3 = RandomForestClassifier(n_estimators=159, random_state=42)
model_4 = XGBClassifier(n_estimators=159, random_state=42)

In [9]:
pd.Series(y_train).value_counts()

Positive    12438
Negative     7660
Name: Sentiment, dtype: int64

### => Data imbalanced so we need use over sampling

In [10]:
smote = BorderlineSMOTE(k_neighbors=3, random_state=42)
X_train_tfidf_res, y_train_encoded_res = smote.fit_resample(X_train_tfidf, y_train_encoded)

In [11]:
pd.Series(y_train_encoded_res).value_counts()

0    12438
1    12438
dtype: int64

In [12]:
model_1.fit(X_train_tfidf_res, y_train_encoded_res)

LogisticRegression(random_state=42, solver='liblinear')

In [13]:
y_pred_1 = model_1.predict(X_test_tfidf)

In [14]:
print(classification_report(y_true=y_test, y_pred=encoder.inverse_transform(y_pred_1), digits=2))

              precision    recall  f1-score   support

    Negative       0.85      0.85      0.85       859
    Positive       0.91      0.90      0.91      1375

    accuracy                           0.88      2234
   macro avg       0.88      0.88      0.88      2234
weighted avg       0.88      0.88      0.88      2234



In [15]:
model_2.fit(X_train_tfidf_res, y_train_encoded_res)

MultinomialNB(force_alpha=True)

In [16]:
y_pred_2 = model_2.predict(X_test_tfidf)

In [17]:
print(classification_report(y_true=y_test, y_pred=encoder.inverse_transform(y_pred_2), digits=2))

              precision    recall  f1-score   support

    Negative       0.86      0.84      0.85       859
    Positive       0.90      0.91      0.91      1375

    accuracy                           0.89      2234
   macro avg       0.88      0.88      0.88      2234
weighted avg       0.89      0.89      0.89      2234



In [19]:
model_3.fit(X_train_tfidf_res, y_train_encoded_res)

RandomForestClassifier(n_estimators=159, random_state=42)

In [20]:
y_pred_3 = model_3.predict(X_test_tfidf)

In [21]:
print(classification_report(y_true=y_test, y_pred=encoder.inverse_transform(y_pred_3), digits=2))

              precision    recall  f1-score   support

    Negative       0.84      0.80      0.82       859
    Positive       0.88      0.90      0.89      1375

    accuracy                           0.86      2234
   macro avg       0.86      0.85      0.85      2234
weighted avg       0.86      0.86      0.86      2234



In [22]:
model_4.fit(X_train_tfidf_res, y_train_encoded_res)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=159, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [23]:
y_pred_4 = model_4.predict(X_test_tfidf)

In [24]:
print(classification_report(y_true=y_test, y_pred=encoder.inverse_transform(y_pred_4), digits=2))

              precision    recall  f1-score   support

    Negative       0.84      0.82      0.83       859
    Positive       0.89      0.91      0.90      1375

    accuracy                           0.87      2234
   macro avg       0.87      0.86      0.86      2234
weighted avg       0.87      0.87      0.87      2234



In [18]:
with open('model_normal_1.pkl', 'wb') as f:
  pickle.dump(model_1, f)
with open('model_normal_2.pkl', 'wb') as f:
  pickle.dump(model_2, f)
with open('tfidf_normal.pkl', 'wb') as f:
  pickle.dump(tfidf, f)
with open('label_encoder_normal.pkl', 'wb') as f:
  pickle.dump(encoder, f)

### Predict text

In [19]:
with open('model_normal_2.pkl', 'rb') as f:
  model_normal_2 = pickle.load(f)

In [20]:
def predict_text(text):
  return encoder.inverse_transform(model_normal_2.predict(tfidf.transform([text])))[0]

In [24]:
predict_text('Đồ ăn ở đây rất ngon')

'Positive'

In [25]:
predict_text('Đồ ăn ở đây rất tệ')

'Negative'

## => Sau khi đã tiền xử lí dữ liệu (xử lí rating, đặt lại khoảng điểm để đánh giá bình luận tích cực hay tiêu cực) thì sẽ đưa dữ liệu vào mô hình để huấn luyện và đánh giá.
## => Nhưng có một điểm lưu ý ở đây chính là dữ liệu ở tập train đang bị mất cân bằng nên phải sử dụng biện pháp over sampling cụ thể là BorderlineSMOTE để cân bằng lại dữ liệu huấn luyện.
## => Sau khi đã huấn luyện xong thì mô hình cho ra đánh giá khá ổn với accuracy đạt được cao nhất là 89% với thuật toán Naive Bayes. Và chúng ta cũng có thể thực hiện kiểm tra một câu xem là tích cực hay tiêu cực bằng cách truyền một câu vào hàm `predict_text()` để mô hình sẽ đưa ra kết luận là Positive (tích cực) hay Negative (tiêu cực).